In [1]:
%cd ..

/home/daryna/pet_projects/ring-customiser-chat-bot


/home/daryna/miniconda3/envs/ring-customizer/lib/python3.11/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [2]:
from pathlib import Path
from dotenv import load_dotenv
from enum import Enum
from operator import itemgetter

from langchain_community.document_loaders import TextLoader
from langchain_openai import OpenAIEmbeddings
from langchain.vectorstores import DocArrayInMemorySearch
from langchain_text_splitters import CharacterTextSplitter
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.schema.output_parser import StrOutputParser
from langchain_core.runnables import RunnableParallel, RunnableLambda
from langchain_openai import ChatOpenAI
from langchain.output_parsers import EnumOutputParser
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.output_parsers import JsonOutputParser

from src.schemas import Ring, SupportRequest

load_dotenv()

True

## Set up retrievers

In [3]:
DATA_DIR = Path('data')
customization_filepath = DATA_DIR / 'customization.md'
faq_filepath = DATA_DIR / 'FAQ.md'

customization_loader = TextLoader(file_path=customization_filepath, encoding='utf-8')
faq_loader = TextLoader(file_path=faq_filepath, encoding='utf-8')

customization_documents = customization_loader.load()
faq_documents = faq_loader.load();

text_splitter = CharacterTextSplitter(chunk_size=200, chunk_overlap=0)
customization_docs = text_splitter.split_documents(customization_documents)
faq_docs = text_splitter.split_documents(faq_documents)

embedding_model = OpenAIEmbeddings()

vectorstore_customizations = DocArrayInMemorySearch.from_documents(
    customization_docs, 
    embedding_model
)
vectorstore_faq = DocArrayInMemorySearch.from_documents(
    faq_docs, 
    embedding_model
)

retriever_customizations = vectorstore_customizations.as_retriever(search_kwargs={'k': 2})
retriever_faq = vectorstore_faq.as_retriever(search_kwargs={'k': 2})

Created a chunk of size 867, which is longer than the specified 200
Created a chunk of size 213, which is longer than the specified 200
Created a chunk of size 248, which is longer than the specified 200
/home/daryna/miniconda3/envs/ring-customizer/lib/python3.11/site-packages/pydantic/_migration.py:283: UserWarning: `pydantic.error_wrappers:ValidationError` has been moved to `pydantic:ValidationError`.
  warnings.warn(f'`{import_path}` has been moved to `{new_location}`.')


## Retrieval with history

In [4]:
def create_prompt(prompt_str, include_history = True):
    messages = [("system", prompt_str)]
    if include_history:
        messages.append(MessagesPlaceholder(variable_name="history"))
    messages.append(("human", "{input}"))

    return ChatPromptTemplate.from_messages(messages)

In [5]:
output_parser = StrOutputParser()
prompt_str = """Answer the question below using the context:

Context:
{context_customizations}
{context_faq}
"""

prompt = create_prompt(prompt_str)

retrieval = RunnableParallel(
    {
        "context_customizations": RunnableLambda(itemgetter("input")) | retriever_customizations, 
        "context_faq": RunnableLambda(itemgetter("input")) | retriever_customizations,
        'input': itemgetter("input"),
        'history': itemgetter("history")
    }
)
llm = ChatOpenAI(model='gpt-3.5-turbo', temperature=0)
chain = retrieval | prompt | llm | output_parser

In [6]:
store = {}

def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]

with_message_history = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="input",
    history_messages_key="history",
)

In [7]:
with_message_history.invoke(
    {"input": "Is there a way to determine the size of the ring for me at home?"},
    config={"configurable": {"session_id": "abc123"}},
)

Parent run 97cf57f4-39c5-4072-b810-f8cfe491ea82 not found for run 2756e292-35ca-4257-ae3b-ea90c41a671a. Treating as a root run.


'Yes, you can determine the size of the ring at home. The size ranges from 4 to 13, and the ring width ranges from 1mm to 8mm. You can measure your finger size using a ring sizer tool or by using a printable ring size chart available online.'

## Router (input classification) chain

In [8]:
class Topic(str, Enum):
    CUSTOMIZATION = 'customization'
    RING = 'ring'
    REQUEST = 'request'
    FAQ = 'faq'

#- If the user is admitting that the request to support is finalized (message "Confirm") classify as "request".

router_prompt_str = """
Given the input and the conversation history classify:

- If the user would like to customize or order a ring ("I would like to order a ring", "I would like to customize a ring" etc.), or being in the process of customizing giving to you a selected options from the customization context classify as "customization".
- If correctness of the customizations is being confirmed (message "Correct") classify as "ring".
- If the user expresses frustration, issues, or needs help beyond FAQ answers (e.g., "I can't find my order," "I need help with my account") or would like to make a direct request or to reach out for a support team OR you just received message "Confirm" classify as "request".
- If the user asks questions covered in the FAQ classify as "faq".

Notice: 
Do not answer the question or make up the answer or question, only return as simple as possible, eithter 'customization', 'ring', 'request' or 'faq' as string without any instruction text, reasoning text, headlines, leading-text or other additional information.

Customizations Context: 
{context_customizations}

Format instructions:
{format_instructions}
Answer:
"""
router_prompt_str = router_prompt_str.replace(
    '{format_instructions}', EnumOutputParser(enum=Topic).get_format_instructions()
)
router_prompt = create_prompt(router_prompt_str, include_history=False)

In [9]:
retrieval = RunnableParallel(
    {
        "context_customizations": RunnableLambda(itemgetter("input")) | retriever_customizations, 
        "context_faq": RunnableLambda(itemgetter("input")) | retriever_faq,
        'input': itemgetter("input"),
        'history': itemgetter("history")
    }
)

router_parser = StrOutputParser()

router_chain = {
    "context_customizations": itemgetter('context_customizations'), 
    "context_faq": itemgetter("context_faq"),
    'input': itemgetter("input"),
    'history': itemgetter("history"),
    'topic': router_prompt | llm | router_parser
}

chain = retrieval | router_chain


In [10]:
store = {}

with_message_history = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="input",
    history_messages_key="history",
)

In [11]:
with_message_history.invoke(
    {"input": "I would like to have a ring: gold, classic, polished, size 5, 2mm, no engraving. Complete"},
    config={"configurable": {"session_id": "abc123"}},
)

Parent run e2ea5fbd-6a91-4cc3-bb92-1afbb55bd8a0 not found for run 4aa679a8-1399-488a-b394-3f60abc320e8. Treating as a root run.
Error in RootListenersTracer.on_chain_end callback: KeyError('output')


{'context_customizations': [Document(page_content='#### Size\nFrom 4 to 13\n\n#### Ring Width\nFrom 1mm to 8mm\n\n#### Engraving\nUp to 20 characters or empty', metadata={'source': 'data/customization.md'}),
  Document(page_content='#### Material\n- Yellow Gold\n- White Gold\n- Platinum\n- Sterling Silver\n- Titanium\n\n#### Style\n-\tClassic\n-\tModern\n-\tVintage\n-\tBohemian\n\n#### Surface\n-\tPolished\n-\tMatte\n-\tHammered\n-\tBrushed', metadata={'source': 'data/customization.md'})],
 'context_faq': [Document(page_content='**Q:** Can I get assistance with my design?\n**A:** Yes, our design team is available to help you create the perfect ring.', metadata={'source': 'data/FAQ.md'}),
  Document(page_content='**Q:** What materials are the rings made from?\n**A:** Our rings are available in gold, silver, platinum, and titanium.', metadata={'source': 'data/FAQ.md'})],
 'input': 'I would like to have a ring: gold, classic, polished, size 5, 2mm, no engraving. Complete',
 'history': [],

## Branch chain for different tasks

In [12]:
ring_customization_prompt_str = """
Your task it to guide the user through the process of a ring customization. Firstly, you give all the possible customizations and their options to the customer. Ask the customer to choose all the customizations from the possible options given in the context. Make sure user fills in all of the possible customizations. Do not allow user to select options on customizations that were not specified in the context. Check if user selects customizations that are not valid and in that case correct them. If user forgets to specify some customizations remind the user to select one of available choises. After all the customizations are collected, show all the customizations and options selected by user once again and ask to confirm the customizations by typing "Correct". If something goes wrong or your struggle to answer customer questions or fullfill customer's request, politely explain that you cannot anser that question and propose to the customer that the question can be passed to the support team. Ask to type "Confirm" in that case.

Context:
{context_customizations}
"""
ring_customization_prompt = create_prompt(ring_customization_prompt_str)
ring_customization_chain = ring_customization_prompt | llm | output_parser

ring_customization_output_prompt_str = """
Your task is to collect the user customizations from the current conversation and format it appropriately.

Output format is JSON with fields:
material
style
surface
size
ring_width
engraving

Notice:
Do not add other fields.
"""
# ring_customization_output_parser = PydanticOutputParser(pydantic_object=Ring)
ring_customization_output_prompt = create_prompt(ring_customization_output_prompt_str)
ring_customization_output_chain = (ring_customization_output_prompt 
                                   | llm.bind(response_format={"type": "json_object"}) 
                                   | JsonOutputParser(pydantic_object=Ring)
                                   )

# support_prompt_str = """
# Your task is to collect information, summarize and extract key details on the request from user based on the current conversation. After the user told what is the request, repeat it for user and ask for the verification on correctness. Ask to type "Confirm" in that case.
# """
# support_prompt = create_prompt(support_prompt_str)
# support_chain = support_prompt | llm | output_parser

support_output_prompt_str = """
Your task is to collect the last user request from the current conversation and format it appropriately.

Output format is JSON with fields:
customer_message
conversation_summary
key_details

Notice:
Do not add other fields.
"""
support_output_prompt = create_prompt(support_output_prompt_str)
support_output_chain = (support_output_prompt 
                        | llm.bind(response_format={"type": "json_object"}) 
                        | JsonOutputParser(pydantic_object=SupportRequest)
                        )

faq_prompt_str = """
Your task is to answer questions based on context. If question is not covered in the context, explain is not covered in our FAQ and propose to the customer that the question can be passed to the support team. Ask to type "Confirm" in that case.

Customizations Context: {context_customizations}
FAQ Context: {context_faq}
"""
faq_prompt = create_prompt(faq_prompt_str)
faq_chain = faq_prompt | llm | output_parser

In [ ]:
def route(info):
    print(info['topic'])
    if "customization" in info["topic"].lower():
        return ring_customization_chain
    elif "ring" in info["topic"].lower():
        return ring_customization_output_chain.with_types(output_type=Ring)
    elif "request" in info["topic"].lower():
        return support_output_chain.with_types(output_type=SupportRequest)
    else:
        return faq_chain

## Start a conversation

In [13]:
chain = retrieval | router_chain | RunnableLambda(route)

store = {}

with_message_history = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="input",
    history_messages_key="history",
)

In [14]:
while True:
    human_input = ''
    while human_input == '':
        human_input = input()
    print('\nHuman:', human_input)
    ai_output = with_message_history.invoke(
        {"input": human_input},
        config={"configurable": {"session_id": "abc123"}},
    )
    print('\nAI:', ai_output)


Parent run 663c8d8b-a50c-44fd-b9a3-6eb66d93b73b not found for run 7c66d42d-9848-4518-9a8c-89354a278c94. Treating as a root run.



Human: Hello, I would like to customize a ring
customization

AI: Great! I can help you with that. Let's start by selecting the customization options for your ring. Here are the possible customizations:

1. Size (From 4 to 13)
2. Ring Width (From 1mm to 8mm)
3. Engraving (Up to 20 characters or empty)
4. Material (Yellow Gold, White Gold, Platinum, Sterling Silver, Titanium)
5. Style (Classic, Modern, Vintage, Bohemian)
6. Surface (Polished, Matte, Hammered, Brushed)

Please choose your preferences for each customization option. Let's start with the first one.


Parent run b8733b01-b8e1-4fb7-82a5-04fa10e7f20b not found for run ba891f8e-d4d5-4a34-8b94-65a72700fd4a. Treating as a root run.



Human: size 5, 1mm, no engraving, classic, matte
customization

AI: Great choices! Let's review your selections:
- Size: 5
- Ring Width: 1mm
- Engraving: None
- Style: Classic
- Surface: Matte

Please confirm if these are correct by typing "Correct".


Parent run f7945185-aae7-49ec-bbfb-369de0e38ed2 not found for run c0a9e9e6-5160-4250-8631-bfd31141deb3. Treating as a root run.


...

Parent run 33a0e6a3-622a-42b2-8601-1d060f1a46b3 not found for run f2c241a1-5c57-4602-a916-cfcab27ee67a. Treating as a root run.



Human: yes, please
request


Error in RootListenersTracer.on_chain_end callback: KeyError('output')



AI: {'customer_message': 'I would like to customize a ring', 'conversation_summary': 'Customer wants to customize a ring with the following specifications: Size 5, Ring Width 1mm, No Engraving, Classic Style, Matte Surface. Customer also wants to know how to determine their ring size.', 'key_details': {'customization_options': {'Size': '5', 'Ring Width': '1mm', 'Engraving': 'None', 'Style': 'Classic', 'Surface': 'Matte'}, 'additional_request': 'Customer wants to know how to determine their ring size'}}


KeyboardInterrupt: Interrupted by user